## First: explore the data
Review the unstructured csv files and answer the following questions with code that supports your conclusions:
- Are there any data quality issues present? **Yes**
  - **Product Table**.
    - Missing Values. All 7 columns contain missing values. `CATEGORY_4` contains the highest number of missing values, 73% missing. 10% of Column `MANUFACTURER` has 'PLACEHOLDER MANUFACTURER' as the value instead of real manufacturer.
    - Relationship between `CATEGORY_2` and `CATEGORY_3` is not following the one-to-many relationship (assuming they should). `CATEGORY_3` 'Soda' has two `CATEGORY_2`, 'Beverages' and 'Carbonated Soft Drinks'. This is the only case identifed in the given table.
    - Some `BARCODE`s have more than one occurance in the table. Examples show one barcode could have two different brands and manufacturers (likely incorrect input, ex.), or it could be just purely duplicated rows.  There are 185 barcodes in total having more than one accurance.
    - Found two brands are not all-uppercase. One example is "Listerine", the table also contains "LISTERINE"
  - **User Table**.
    - Missing Values. All columns have missing values except for `ID`. `LANGUAGE` has the most missing values, 30% missing.
    - `GENDER` data is noisy. 'My gender isn't listed' and 'not_listed' are essentially the same meaning, same situation with 'Prefer not to say' and 'prefer_not_to_say'
    - Only 1% users in txn table can be found in user table
  - **Transaction Table**
    - Seemingly no missing values excpet for `BARCODE`. However, `FINAL_QUANTITY` is stored as string because instead of a numeric representaion, 25% of its value is 'zero'. Similar situation with `FINAL_SALE`, 25% of its value is ' '. If removing all the 'zero's and ' ' in these two columns, only 50% of the data will be preserved.
    - 94 rows showing the `SCAN_DATE` is before the `PURCHASE_DATE`. Assuming the scan sate is the date customer scan the receipt (after purchasing), these 94 rows don't make logical sense. Depends on the IT implementation of this data collection, it could be the timezones not aligning.
    - 8 rows have barcode of '-1'
    - 171 duplicate rows. They could be valid input in the scenario of multiple lines of the same item (barcode).
- Are there any fields that are challenging to understand?
    - The only thing that looks a bit unfamiliar to me is 'es-419' in the `LANGUAGE` column of the user table. After some searching, seems it represnet the Spanish as it is spoken in Latin America and the Caribbean.

We recommend using SQL or python and data visualization to examine the data.

In [2]:
import pandas as pd
import numpy as np

In [3]:
# read data and check number of rows
df_prod = pd.read_csv("PRODUCTS_TAKEHOME.csv")
df_txn = pd.read_csv("TRANSACTION_TAKEHOME.csv")
df_user = pd.read_csv("USER_TAKEHOME.csv")

len(df_prod), len(df_txn), len(df_user)

(845552, 50000, 100000)

### Explore product data

In [3]:
#check what the columns stand for
df_prod.head(5)

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11


In [6]:
df_prod.dtypes

CATEGORY_1       object
CATEGORY_2       object
CATEGORY_3       object
CATEGORY_4       object
MANUFACTURER     object
BRAND            object
BARCODE         float64
dtype: object

In [8]:
# check missing values and the top values
df_prod.describe(include='all')

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
count,845441,844128,784986,67459,619078,619080,8.415270e+05
unique,27,121,344,127,4354,8122,NaN
top,Health & Wellness,Candy,Confection Candy,Lip Balms,PLACEHOLDER MANUFACTURER,REM BRAND,NaN
freq,512695,121036,56965,9737,86902,20813,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,6.016109e+11
std,NaN,NaN,NaN,NaN,NaN,NaN,1.022530e+12
min,NaN,NaN,NaN,NaN,NaN,NaN,1.850000e+02
25%,NaN,NaN,NaN,NaN,NaN,NaN,7.124923e+10
50%,NaN,NaN,NaN,NaN,NaN,NaN,6.344185e+11
75%,NaN,NaN,NaN,NaN,NaN,NaN,7.683955e+11


In [23]:
# check if CATEGORY_1 ~ CATEGORY_4 follow the product hierarchy logic (one-to-many)
c4_to_c3 = df_prod.groupby('CATEGORY_4')['CATEGORY_3'].nunique()

c3_to_c2 = df_prod.groupby('CATEGORY_3')['CATEGORY_2'].nunique()

c2_to_c1 = df_prod.groupby('CATEGORY_2')['CATEGORY_1'].nunique()

(c4_to_c3 <= 1).all(),(c3_to_c2 <= 1).all(),(c2_to_c1 <= 1).all()

(True, False, True)

In [21]:
# check how often the c3_to_c2 hierarchy is not followed and find examples
c3_to_c2.sort_values(ascending=False)

CATEGORY_3
Soda                               2
Yoga & Pilates                     1
Facial Lotion & Moisturizer        1
Ear Plugs                          1
Emergency Tools & Kits             1
                                  ..
Packaged Cherries                  1
Packaged Mixed Fruit & Cocktail    1
Packaged Peaches                   1
Packaged Pears                     1
AV Carts                           1
Name: CATEGORY_2, Length: 344, dtype: int64

In [24]:
df_prod.query("CATEGORY_3=='Soda'")[["CATEGORY_2","CATEGORY_3"]].drop_duplicates()

,CATEGORY_2,CATEGORY_3
1315,Beverages,Soda
4704,Carbonated Soft Drinks,Soda


In [26]:
# validate if product data is on barcode level. If not, how often
len(df_prod[['BARCODE']].dropna()),len(df_prod[['BARCODE']].dropna().drop_duplicates())

(841527, 841342)

In [34]:
barcode_cnt = df_prod.groupby("BARCODE")["BARCODE"].count()
len(barcode_cnt[barcode_cnt>1])

185

In [36]:
barcode_cnt.sort_values(ascending=False)

BARCODE
3.484708e+06    2
2.015908e+07    2
3.400906e+06    2
3.400807e+06    2
3.400708e+06    2
               ..
7.289256e+11    1
7.289256e+11    1
7.289256e+11    1
7.289256e+11    1
1.850000e+02    1
Name: BARCODE, Length: 841342, dtype: int64

In [39]:
# check duplicated barcode example
df_prod[df_prod["BARCODE"].isin([3484708,3400906])]

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
160743,Snacks,Candy,Mints,NaN,THE HERSHEY COMPANY,ICE BREAKERS,3400906.0
594684,Snacks,Candy,Gum,NaN,THE HERSHEY COMPANY,ICE BREAKERS,3484708.0
717446,Snacks,Candy,Gum,NaN,GENERAL LICENSED IP MANUFACTURER,DISNEY,3484708.0
732002,Snacks,Candy,Mints,NaN,THE HERSHEY COMPANY,ICE BREAKERS,3400906.0


In [87]:
# check MANUFACTURER format
df_prod[['MANUFACTURER']][~df_prod['MANUFACTURER'].fillna('').astype(str).str.isupper()].dropna()['MANUFACTURER'].unique()

array(['5.11'], dtype=object)

In [88]:
# check BRAND format
df_prod[['BRAND']][~df_prod['BRAND'].fillna('').astype(str).str.isupper()].dropna()['BRAND'].unique()

array(['Listerine', "Kellogg's"], dtype=object)

In [92]:
df_prod.query('BRAND=="LISTERINE"')

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
2654,Health & Wellness,Oral Care,Mouthwash,NaN,JOHNSON & JOHNSON,LISTERINE,7.527984e+11
5057,Health & Wellness,Oral Care,Floss,NaN,JOHNSON & JOHNSON,LISTERINE,1.915652e+11
5268,Health & Wellness,Oral Care,Mouthwash,NaN,JOHNSON & JOHNSON,LISTERINE,5.446726e+11
5819,Health & Wellness,Oral Care,Breath Strips & Sprays,NaN,JOHNSON & JOHNSON,LISTERINE,7.047983e+11
6231,Health & Wellness,Oral Care,Mouthwash,NaN,JOHNSON & JOHNSON,LISTERINE,7.086032e+11
...,...,...,...,...,...,...,...
834758,Health & Wellness,Oral Care,Toothpaste,NaN,JOHNSON & JOHNSON,LISTERINE,3.125474e+11
835731,Health & Wellness,Oral Care,Breath Strips & Sprays,NaN,JOHNSON & JOHNSON,LISTERINE,6.441351e+11
836110,Health & Wellness,Oral Care,Mouthwash,NaN,JOHNSON & JOHNSON,LISTERINE,1.915657e+11
840996,Health & Wellness,Oral Care,Breath Strips & Sprays,NaN,JOHNSON & JOHNSON,LISTERINE,7.385534e+11


In [97]:
df_prod[(df_prod['BRAND'].fillna('').str.contains("KELLOGG")) | (df_prod['BRAND']=="Kellogg's")].BRAND.unique()

array(["KELLOGG'S CEREAL VARIETY PACKS", "KELLOGG'S CORN FLAKES",
       "KELLOGG'S SNACKS VARIETY PACKS", "Kellogg's"], dtype=object)

### Explore user data

In [41]:
#check what the columns stand for
df_user.head(5)

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female


In [42]:
# check missing values and the top values
df_user.describe(include='all')

,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
count,100000,100000,96325,95188,69492,94108
unique,100000,99942,54721,52,2,11
top,6388d573ae4fd936d3ed3254,2021-12-05 01:32:56.000 Z,1970-01-01 00:00:00.000 Z,TX,en,female
freq,1,2,1272,9028,63403,64240


In [77]:
df_user.dtypes

ID              object
CREATED_DATE    object
BIRTH_DATE      object
STATE           object
LANGUAGE        object
GENDER          object
dtype: object

In [44]:
df_user.groupby('GENDER')['GENDER'].count()

GENDER
My gender isn't listed        5
Non-Binary                   34
Prefer not to say             1
female                    64240
male                      25829
non_binary                  473
not_listed                  180
not_specified                28
prefer_not_to_say          1350
transgender                1772
unknown                     196
Name: GENDER, dtype: int64

### Explore transaction data

In [48]:
#check what the columns stand for
df_txn.head(5)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


In [46]:
# check missing values and the top values or distribution
df_txn.describe(include='all')

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
count,50000,50000,50000,50000,50000,4.423800e+04,50000,50000
unique,24440,89,24440,954,17694,NaN,87,1435
top,bedac253-2256-461b-96af-267748e6cecf,2024-06-15,2024-09-08 20:00:42.348 Z,WALMART,64e62de5ca929250373e6cf5,NaN,1.00,
freq,12,774,12,21326,22,NaN,35698,12500
mean,NaN,NaN,NaN,NaN,NaN,1.715863e+11,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,3.269219e+11,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,-1.000000e+00,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,3.077212e+10,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,5.210004e+10,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,8.536765e+10,NaN,NaN


In [63]:
df_txn.dtypes

RECEIPT_ID                 object
PURCHASE_DATE              object
SCAN_DATE                  object
STORE_NAME                 object
USER_ID                    object
BARCODE                   float64
FINAL_QUANTITY             object
FINAL_SALE                 object
FINAL_QUANTITY_cleaned    float64
FINAL_SALE_cleaned        float64
dtype: object

In [49]:
len(df_txn.query("FINAL_QUANTITY=='zero'"))

12500

In [55]:
# simple cleaning by settinng "zero" quantity and '' sale to na, converting them to 0 would not be making sense in retail
df_txn['FINAL_QUANTITY_cleaned'] = df_txn['FINAL_QUANTITY'].replace('zero', np.nan).astype(float)
df_txn['FINAL_SALE_cleaned'] = df_txn['FINAL_SALE'].replace(' ', np.nan).astype(float)

In [64]:
# check if purchase date and scan date makes logical sense
df_txn['date_delta'] = (pd.to_datetime(df_txn['SCAN_DATE']).dt.date - pd.to_datetime(df_txn['PURCHASE_DATE']).dt.date).dt.days

In [65]:
# check again
df_txn.describe(include='all')

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,FINAL_QUANTITY_cleaned,FINAL_SALE_cleaned,date_delta
count,50000,50000,50000,50000,50000,4.423800e+04,50000,50000,37500.000000,37500.000000,50000.000000
unique,24440,89,24440,954,17694,NaN,87,1435,NaN,NaN,NaN
top,bedac253-2256-461b-96af-267748e6cecf,2024-06-15,2024-09-08 20:00:42.348 Z,WALMART,64e62de5ca929250373e6cf5,NaN,1.00,,NaN,NaN,NaN
freq,12,774,12,21326,22,NaN,35698,12500,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,1.715863e+11,NaN,NaN,1.085902,4.576384,2.016720
std,NaN,NaN,NaN,NaN,NaN,3.269219e+11,NaN,NaN,2.056250,6.625663,3.385704
min,NaN,NaN,NaN,NaN,NaN,-1.000000e+00,NaN,NaN,0.010000,0.000000,-1.000000
25%,NaN,NaN,NaN,NaN,NaN,3.077212e+10,NaN,NaN,1.000000,1.790000,0.000000
50%,NaN,NaN,NaN,NaN,NaN,5.210004e+10,NaN,NaN,1.000000,3.000000,1.000000
75%,NaN,NaN,NaN,NaN,NaN,8.536765e+10,NaN,NaN,1.000000,5.190000,3.000000


In [59]:
# how many non-null quantity and non-null sales left
len(df_txn[(df_txn["FINAL_QUANTITY_cleaned"].notnull())&(df_txn["FINAL_SALE_cleaned"].notnull())])

25000

In [66]:
df_txn.query("date_delta<0")

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,FINAL_QUANTITY_cleaned,FINAL_SALE_cleaned,date_delta
51,008c1dcc-0f96-4b04-98c8-2a2bb63ef89d,2024-07-21,2024-07-20 19:54:23.133 Z,WALMART,5dc24cdb682fcf1229d04bd6,6.811312e+11,zero,3.18,NaN,3.18,-1
455,04a320ed-2903-45e5-8fd7-6eaf08daef32,2024-06-29,2024-06-28 11:03:31.783 Z,DOLLAR GENERAL STORE,62855f67708670299a658035,4.900002e+10,zero,6.00,NaN,6.00,-1
494,05023b3d-5f83-47a7-a17c-8e8521d0bc94,2024-09-08,2024-09-07 22:22:29.903 Z,SHOP RITE,666a43c77c0469953bfd9ae0,6.414404e+10,2.00,,2.0,NaN,-1
675,06ce3da3-a588-4c37-93b4-0b6d11e42704,2024-06-22,2024-06-21 12:34:15.665 Z,BIG LOTS,646f6ffb7a342372c858487e,3.111117e+11,zero,4.05,NaN,4.05,-1
870,08d0e78f-3e63-40a3-8eb0-73fdf76da52c,2024-06-22,2024-06-21 20:50:01.298 Z,DOLLAR GENERAL STORE,664cafb6e04f743a096a837e,7.680828e+10,2.00,,2.0,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...
46034,08d0e78f-3e63-40a3-8eb0-73fdf76da52c,2024-06-22,2024-06-21 20:50:01.298 Z,DOLLAR GENERAL STORE,664cafb6e04f743a096a837e,7.680828e+10,2.00,3.50,2.0,3.50,-1
46539,718aa730-b62f-4e18-8dba-1d7105dac341,2024-09-05,2024-09-04 20:14:00.374 Z,WALMART,5e0f561efa890112094202ad,7.339001e+10,1.00,3.56,1.0,3.56,-1
46941,af2b818f-4a92-4e98-958c-65f2ce0b271d,2024-06-15,2024-06-14 10:57:23.892 Z,DOLLAR GENERAL STORE,64de6465516348066e7c5690,8.175130e+11,1.00,3.83,1.0,3.83,-1
47653,72bb7b71-d958-4a46-ae62-43abdeb0e693,2024-06-15,2024-06-14 19:55:56.672 Z,WALMART,649726ea127ddb5d7f0004dc,3.485611e+10,1.00,2.05,1.0,2.05,-1


In [68]:
df_txn.query("BARCODE==-1")

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE,FINAL_QUANTITY_cleaned,FINAL_SALE_cleaned,date_delta
169,01cdaa22-a912-496b-8287-87cd9e6b3b1a,2024-07-16,2024-07-17 06:37:09.218 Z,CARRS,6160f71f28dab1659e8b5616,-1.0,zero,5.00,NaN,5.00,1
6010,3d539304-a2d1-47fd-846e-799cc8d1d15d,2024-07-12,2024-07-16 21:05:18.891 Z,DOLLAR TREE STORES INC,6363a9e6d8987029d637d6db,-1.0,1.00,,1.0,NaN,4
10457,6b9ee775-e895-4133-8b84-fbafbe3bd20d,2024-07-11,2024-07-13 13:22:38.638 Z,CVS,63a4ef6bb5f32149b9d86e66,-1.0,zero,3.00,NaN,3.00,2
15621,9f624eb5-bd86-44e6-aaf2-b59339b75c85,2024-07-13,2024-07-13 16:56:58.611 Z,CVS,62893be61ec71241a98013c3,-1.0,zero,0.46,NaN,0.46,0
25312,6b9ee775-e895-4133-8b84-fbafbe3bd20d,2024-07-11,2024-07-13 13:22:38.638 Z,CVS,63a4ef6bb5f32149b9d86e66,-1.0,1.00,3.00,1.0,3.00,2
28894,9f624eb5-bd86-44e6-aaf2-b59339b75c85,2024-07-13,2024-07-13 16:56:58.611 Z,CVS,62893be61ec71241a98013c3,-1.0,1.00,0.46,1.0,0.46,0
42636,3d539304-a2d1-47fd-846e-799cc8d1d15d,2024-07-12,2024-07-16 21:05:18.891 Z,DOLLAR TREE STORES INC,6363a9e6d8987029d637d6db,-1.0,1.00,1.25,1.0,1.25,4
45823,01cdaa22-a912-496b-8287-87cd9e6b3b1a,2024-07-16,2024-07-17 06:37:09.218 Z,CARRS,6160f71f28dab1659e8b5616,-1.0,1.00,5.00,1.0,5.00,1


In [69]:
len(df_txn.drop_duplicates())

49829

### Second: provide SQL queries
Answer three of the following questions with at least one question coming from the closed-ended and one from the open-ended question set. Each question should be answered using one query.
- Closed-ended questions:
 - What are the top 5 brands by receipts scanned among users 21 and over?
   - NERDS CANDY, DOVE, TRIDENT, SOUR PATCH KIDS, MEIJER, HERSHEY'S, GREAT VALUE, COCA-COLA. The last 6 brands are tied.
   - This result is highly unreliable due to the data issue we see. 99% scanned users not having birth date info.
 - What are the top 5 brands by sales among users that have had their account for at least six months?
 - What is the percentage of sales in the Health & Wellness category by generation?
   - 47% from Baby Boomers (yr 1946-1964), 22% from Gen X, 31% from Millennials
   - This result is highly unreliable due to the data issue we see. 99% scanned users not having birth date info.
- Open-ended questions: for these, make assumptions and clearly state them when answering the question.
 - Who are Fetch’s power users?
 - Which is the leading brand in the Dips & Salsa category?
   - TOSTITOS. There are many metrics can be used to define what can be considered as "leading". To answer this question, I looked into total number of scans, store coverage, number of user scanned.
 - At what percent has Fetch grown year over year?

In [4]:
# Use pandasql to write sql
from pandasql import sqldf

# SQL interface
pysqldf = lambda q: sqldf(q, globals())

#### Queries

In [34]:
# Check how manny user ID in txn table can be found in user table.
sql_check_User = """
SELECT
COUNT(DISTINCT A.USER_ID) A_ID,
COUNT(DISTINCT B.ID) B_ID
FROM df_txn A
LEFT JOIN df_user B
 ON A.USER_ID = B.ID
"""

pysqldf(sql2)

,A_ID,B_ID
0,17694,91


In [115]:
# top 5 brands by receipts scanned among users 21 and over. (pulling top 10 to check if there are ties)
sql1 = """
SELECT C.BRAND, COUNT(DISTINCT A.RECEIPT_ID) AS RECEIPT_CNT
FROM df_txn A
INNER JOIN df_user B
 ON A.USER_ID = B.ID
INNER JOIN df_prod C -- assuming the 185 duplicate barcodes will not impacting the top 5 result. Otherwise, will have to dedup before join on barcode
 ON A.BARCODE = C.BARCODE
WHERE CAST(strftime('%Y', 'now') AS INTEGER) - CAST(strftime('%Y', B.BIRTH_DATE) AS INTEGER) >=21 -- simple age calc by only consider year
AND C.BRAND IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;
"""

pysqldf(sql1)

,BRAND,RECEIPT_CNT
0,NERDS CANDY,3
1,DOVE,3
2,TRIDENT,2
3,SOUR PATCH KIDS,2
4,MEIJER,2
5,HERSHEY'S,2
6,GREAT VALUE,2
7,COCA-COLA,2
8,UTZ,1
9,TWIX,1


In [118]:
#What is the percentage of sales in the Health & Wellness category by generation?
# Define generations using source found on https://www.purdueglobal.edu/education-partnerships/generational-workforce-differences-infographic/
sql2 = """
SELECT 
CASE WHEN CAST(strftime('%Y', B.BIRTH_DATE) AS INTEGER) BETWEEN 1925 AND 1945 THEN "Traditionalists"
     WHEN CAST(strftime('%Y', B.BIRTH_DATE) AS INTEGER) BETWEEN 1946 AND 1964 THEN "Baby Boomers"
     WHEN CAST(strftime('%Y', B.BIRTH_DATE) AS INTEGER) BETWEEN 1965 AND 1980 THEN "Generation X"
     WHEN CAST(strftime('%Y', B.BIRTH_DATE) AS INTEGER) BETWEEN 1981 AND 2000 THEN "Millennials"
     WHEN CAST(strftime('%Y', B.BIRTH_DATE) AS INTEGER) BETWEEN 2001 AND 2020 THEN "Generation Z"
     ELSE "Other" END AS GENERATION,
SUM(FINAL_SALE_cleaned) SALES,
SUM(FINAL_SALE_cleaned)/(SUM(SUM(FINAL_SALE_cleaned)) OVER()) AS SALES_PERC

FROM df_txn A
INNER JOIN df_user B
 ON A.USER_ID = B.ID
INNER JOIN df_prod C -- assuming the 185 duplicate barcodes impact is low to none. Otherwise, will have to dedup before join on barcode
 ON A.BARCODE = C.BARCODE
WHERE 1=1
AND CATEGORY_1 = "Health & Wellness"
GROUP BY 1
"""

pysqldf(sql2)

,GENERATION,SALES,SALES_PERC
0,Baby Boomers,89.03,0.469419
1,Generation X,41.50,0.218813
2,Millennials,59.13,0.311768


In [24]:
# Which is the leading brand in the Dips & Salsa category?
sql3 = """
SELECT C.BRAND,

COUNT(DISTINCT A.RECEIPT_ID) RECEIPT_CNT,
COUNT(DISTINCT A.STORE_NAME) STORE_CNT,
COUNT(DISTINCT A.USER_ID) USER_CNT,
CAST(COUNT(DISTINCT A.RECEIPT_ID) AS FLOAT)/COUNT(DISTINCT A.USER_ID) AVG_SCAN_PER_USER

FROM df_txn A
--INNER JOIN df_user B
-- ON A.USER_ID = B.ID
INNER JOIN df_prod C -- assuming the 185 duplicate barcodes impact is low. Otherwise, will have to dedup before join on barcode
 ON A.BARCODE = C.BARCODE
WHERE BRAND IS NOT NULL
AND CATEGORY_2 = "Dips & Salsa"
GROUP BY 1
ORDER BY 2 DESC
"""

pysqldf(sql3)

,BRAND,RECEIPT_CNT,STORE_CNT,USER_CNT,AVG_SCAN_PER_USER
0,TOSTITOS,36,16,35,1.028571
1,PACE,24,8,24,1.000000
2,FRITOS,19,9,19,1.000000
3,DEAN'S DAIRY DIP,17,4,17,1.000000
4,MARKETSIDE,16,1,16,1.000000
5,HELUVA GOOD!,15,3,15,1.000000
6,FRESHNESS GUARANTEED,12,1,12,1.000000
7,MARZETTI,11,7,11,1.000000
8,HIDDEN VALLEY,10,2,10,1.000000
9,SABRA,9,6,9,1.000000


In [122]:
# At what percent has Fetch grown year over year?
#  Can't get YoY trend when there is only one year data.
sql4 = """
SELECT CAST(strftime('%Y', A.SCAN_DATE) AS INTEGER) SCAN_YEAR,

COUNT(DISTINCT A.RECEIPT_ID) RECEIPT_CNT,
COUNT(DISTINCT A.STORE_NAME) STORE_CNT,
COUNT(DISTINCT A.USER_ID) USER_CNT,
COUNT(DISTINCT C.CATEGORY_2) CATEGORY_2_CNT,
COUNT(DISTINCT A.RECEIPT_ID)/COUNT(DISTINCT A.USER_ID) AVG_SCAN_PER_USER,
COUNT(DISTINCT A.STORE_NAME)/COUNT(DISTINCT A.USER_ID) AVG_STORE_PER_USER,
COUNT(DISTINCT C.CATEGORY_2)/COUNT(DISTINCT A.USER_ID) AVG_C2_PER_USER

FROM df_txn A
--INNER JOIN df_user B
-- ON A.USER_ID = B.ID
LEFT JOIN df_prod C -- assuming the 185 duplicate barcodes impact is low. Otherwise, will have to dedup before join on barcode
 ON A.BARCODE = C.BARCODE
GROUP BY 1
ORDER BY 1
"""

pysqldf(sql4)

,SCAN_YEAR,RECEIPT_CNT,STORE_CNT,USER_CNT,CATEGORY_2_CNT,AVG_SCAN_PER_USER,AVG_STORE_PER_USER,AVG_C2_PER_USER
0,2024,24440,954,17694,50,1,0,0


In [32]:
# At what percent has Fetch grown month over month?
#  month in 2024
sql5 = """
SELECT CAST(strftime('%m', A.SCAN_DATE) AS INTEGER) SCAN_MONTH,
COUNT(DISTINCT substr(A.SCAN_DATE, 1,10)) DAYS_IN_MONTH,

COUNT(DISTINCT A.RECEIPT_ID) RECEIPT_CNT,
COUNT(DISTINCT A.STORE_NAME) STORE_CNT,
COUNT(DISTINCT A.USER_ID) USER_CNT,
COUNT(DISTINCT C.CATEGORY_2) CATEGORY_2_CNT,
CAST(COUNT(DISTINCT A.RECEIPT_ID) AS FLOAT)/COUNT(DISTINCT substr(A.SCAN_DATE, 1,10)) AVG_SCAN_PER_DAY


FROM df_txn A
LEFT JOIN df_prod C -- assuming the 185 duplicate barcodes impact is low. Otherwise, will have to dedup before join on barcode
 ON A.BARCODE = C.BARCODE
GROUP BY 1
ORDER BY 1
"""

pysqldf(sql5)

,SCAN_MONTH,DAYS_IN_MONTH,RECEIPT_CNT,STORE_CNT,USER_CNT,CATEGORY_2_CNT,AVG_SCAN_PER_DAY
0,6,19,4633,310,4334,44,243.842105
1,7,31,9197,502,8063,46,296.677419
2,8,31,8457,456,7440,47,272.806452
3,9,8,2153,195,2092,42,269.125000


### Third: communicate with stakeholders
Construct an email or slack message that is understandable to a product or business leader who is not familiar with your day-to-day work. Summarize the results of your investigation. Include:
- Key data quality issues and outstanding questions about the data
- One interesting trend in the data
 - Use a finding from part 2 or come up with a new insight
- Request for action: explain what additional help, info, etc. you need to make sense of the data and resolve any outstanding issues

In [ ]:
"""
Hi xxx,

I'm Holly. Sr data analyst in Beverage team. I am working on understanding and cleaning receipt scan data to provide helpful insight.

I’ve finished the initial data exploration and EDA, wanted to share some key findings, along with a few data quality issues we need to address.

Key Findings:
- Top 5 Brands by Receipts Scanned (Users 21 and Over):

NERDS CANDY, DOVE, TRIDENT, SOUR PATCH KIDS, and MEIJER (with HERSHEY'S, GREAT VALUE, and COCA-COLA tied).

- Sales by Generation in Health & Wellness:

47% from Baby Boomers, 22% from Gen X, and 31% from Millennials.

- Leading Brand in Dips & Salsa:

TOSTITOS is the clear leader based on scans, store coverage, and user engagement.

*Please take a grain of salt of using this insights as the data quality is concerning.


Data Quality Issues:
- I see lot of missing values and some duplications, which could impact our capbility of providing actional/reliable insights. 
- Only 1% scanned user can be found in the user table
Do you know if we can get in touch with the data owners to get more understanding on the data?

Next Steps:
Can we get the transaction data over at least 2 years so we can analyze the YoY trend? 
It's bit challenge to get any meaningful trend analysis with only 3 months data.

Can I also get some clarification on how to define power users and year-over-year growth.

Let me know if you need any further details or would like to discuss!

Best regards,
"""